In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# EXPLORATORY DATA ANALYSIS

## GENERAL INFO

In [ ]:
telco = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
telco.head()

In [ ]:
telco.info()

* There are 7043 rows of customer data.
* Each data has 19 features (excluding the CustomerID), 3 of them is numerical and all others are categorical variable.
* Features can be divided by 3 categories:
    * Customer demographic info – gender, senior citizen, partner and dependent
    * Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, streaming TV and streaming movies
    * Customer account information – tenure(how long they’ve been a customer), contract, payment method, paperless billing, monthly charges, and total charges
* Our target is the Churn column which shows whether the customer left within the last month.

## MISSING DATA

* It seems this dataset doesn't have any missing values. But there is a problem in **TotalCharges** column, the values are numeric but it is of type object. If we look deep, we can see that there are 11 rows that are empty, so we will drop these 11 rows and convert this column to float.

In [ ]:
len(telco[telco['TotalCharges'] == " "])

In [ ]:
telco.drop(telco[telco['TotalCharges'] == " "].index, inplace=True)
telco['TotalCharges'] = telco['TotalCharges'].astype('float64')

In [ ]:
# Calculate the missing value counts for each column
missing = telco.isnull().sum()
# Show the number of columns that have missing values
missing[missing > 0].count()

## TARGET VARIABLE (CHURN)

* For simplicity, let's call the customers who left within the last month as **Churning Customers** and others as **Non-Churning Customers**
* Let's look at Churning and Non-Churning Customer counts.

In [ ]:
sns.countplot(data=telco, x="Churn")
plt.title('Churn Distribution')
plt.show()

In [ ]:
print("Churn Percentage")
print(telco['Churn'].value_counts(normalize=True)* 100)

* Only 26.5% of customers have left within the last month which shows us that we are dealing with **imbalanced data**. So we should be aware of this situation especially in choosing right evaluation metric, i.e. Accuracy may not be sufficient metric and may even mislead us.

## RELATIONSHIP BETWEEN TARGET VARIABLE AND FEATURES

### NUMERICAL FEATURES

* Let's look at the probability density distribution of these features.

In [ ]:
num_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
fig, ax = plt.subplots(figsize=(15, 9))
ax = [plt.subplot(221), plt.subplot(222), plt.subplot(223)]
plt.subplots_adjust(hspace=0.3)
for i in range(0, len(ax)):
    ax[i].set_title("Distribution of " + num_features[i] + " by Churn")
    ax[i].set_ylabel('Density')
    ax[i].set_xlabel(num_features[i])
    sns.kdeplot(telco[telco['Churn'] == 'Yes'][num_features[i]], color= 'red', label= 'Churn: Yes', ax=ax[i])
    sns.kdeplot(telco[telco['Churn'] == 'No'][num_features[i]], color= 'blue', label= 'Churn: No', ax=ax[i])
plt.show()

OBSERVATIONS
* Most of the Churning Customers leaves in early months of the contract.
* The churning rate increases as the monthly charges increase.
* Totalcharge distribution is the result of the 2 observations above, i.e. it is the result of Tenure and monthly charges.

### CATEGORICAL FEATURES

In [ ]:
# This function plots the churn percentage for each category of given feature 
def plotFeature(feature, xoffset, yoffset):
    df = telco.groupby(feature)["Churn"].value_counts().to_frame()
    df = df.rename({"Churn": "Count"}, axis=1).reset_index()
    df["Percentage"] = df["Count"]/len(telco)*100
    
    ax = sns.barplot(x=feature, y="Percentage", hue="Churn", data=df)
    ax.set_title("Churn percentages for each " + feature)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2 - xoffset,
                height - yoffset,
                '{:1.1f}%'.format(height),
                weight='bold', color='white', size = 12) 

#### GENDER - SENIORCITIZEN

In [ ]:
fig, ax = plt.subplots(figsize=(13,5))
plt.subplot(121)
plotFeature("gender", 0.12, 4.5)
plt.subplot(122)
plotFeature("SeniorCitizen", 0.11, 4.5)
plt.show()

* Gender distribution is homogenious and their churn rate is also same, so it is not an indicative of churn.
* Although Senior citizens are only the 16% of all customers their churn rate is much higher than non-seniors.

#### PARTNER - DEPENDENTS

In [ ]:
fig, ax = plt.subplots(figsize=(13,5))
plt.subplot(121)
plotFeature("Partner", 0.12, 4)
plt.subplot(122)
plotFeature("Dependents", 0.12, 3.5)
plt.show()

* Customers that doesn't have partners have moderately higher churn rate
* Customers without dependents have much higher churn rate

#### PHONE, INTERNET SERVICE, MULTIPLE LINES

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
plt.subplot(131)
plotFeature("PhoneService", 0.12, 3)
plt.subplot(132)
plotFeature("InternetService", 0.18, 2)
plt.subplot(133)
plotFeature("MultipleLines", 0.18, 2)
plt.show()

* Customers without internet have a low churn rate and customers with fiberoptic internet have high churn rate
* Having a Phone Service or Multiple Lines doesn't seem to influence churn

#### ONLINE SECURITY, ONLINE BACKUP, DEVICE PROTECTION

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
plt.subplot(131)
plotFeature("OnlineSecurity", 0.18, 2)
plt.subplot(132)
plotFeature("OnlineBackup", 0.18, 2)
plt.subplot(133)
plotFeature("DeviceProtection", 0.18, 2)
plt.show()

* Customers that doesn't have these 3 services (Online Security, Online Backup and Device Protection) are morely likely to churn

#### TECH SUPPORT, STREAMING TV, STREAMING MOVIES

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
plt.subplot(131)
plotFeature("TechSupport", 0.18, 1.5)
plt.subplot(132)
plotFeature("StreamingTV", 0.18, 1.5)
plt.subplot(133)
plotFeature("StreamingMovies", 0.18, 1.5)
plt.show()

* Customers that doesn't have a tech support are more likely to churn
* Streaming service is not predictive for churn

#### CONTRACT - PAPERLESS BILLING - PAYMENT METHOD

In [ ]:
fig, ax = plt.subplots(figsize=(16,10))
plt.subplots_adjust(hspace=0.2)
plt.subplot(221)
plotFeature("Contract", 0.16, 1.5)
plt.subplot(222)
plotFeature("PaperlessBilling", 0.1, 2)
plt.subplot(223)
plotFeature("PaymentMethod", 0.18, 1.5)
plt.show()

* Intuitively short term contracts have much higher churn rate
* Customers with paperless billing have high churn rate
* Payment method of electronic check shows much higher churn rate than others

### OUTLIER CHECK

In [ ]:
# Look at the IQR for each numerical feature
for i in num_features:
    Q1 = telco[i].quantile(0.25)
    Q3 = telco[i].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR
    print(((telco[i] < lower_limit) | (telco[i] > upper_limit)).any())

* Using IQR method, we didn't detect any outliers in numerical features

### FEATURE ENGINEERING

* Drop the ID column as we don't need it

In [ ]:
telco.drop('customerID', axis=1, inplace=True)

* Generate Dummy variables from categorial features.

In [ ]:
# Create dummy variable for gender but use only one of the column
telco = pd.get_dummies(telco, columns=['gender'], drop_first=True)
# For the features below, we can map Yes/No to binary variables of 0/1
vars1 = ['Churn', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
map1 = {'No': 0, 'Yes': 1}
for i in vars1:
    telco[i] = telco[i].map(map1).astype('int')
# The features below have more than 2 category so we used one-hot encoding
vars2 = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
         'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']
telco = pd.get_dummies(telco, columns=vars2)
telco.shape

* Apply Scaling to numerical features as some of the Machine Learning models need it.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
telco[num_features] = scaler.fit_transform(telco[num_features])

* Correlation of Churn with all features

In [ ]:
plt.figure(figsize=(15,5))
plt.title("Correlation of Features with the Churn")
telco.corr()['Churn'].drop(index='Churn').sort_values(ascending=False).plot(kind='bar')
plt.show()

* As we observe above, the monthly contracts, absence of some services (online security, techsupport) are positively correlated with Churn. On the other hand, tenure, yearly contracts and absence of internet service are negatively correlated.

# MODELING

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import plot_confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
X = telco.drop(["Churn"], axis=1)
y = telco["Churn"]

In [ ]:
# Applied stratified sampling since our data is imbalanced
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=10)

In [ ]:
# Instantiate a dataframe that keeps track of the model results
all_scores = pd.DataFrame()

### HELPER FUNCTIONS

In [ ]:
# This function takes the model, features and actual target values, makes prediction and then prints different metric scores
def model_predictions(name, model, X, y, title):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(y, y_pred, average='binary')
    scores = pd.DataFrame(columns=['Accuracy','Precision','Recall','F1'])
    scores.loc[name] = [accuracy, precision, recall, f1]
    print('----------------------------------------')
    print(title)
    print(scores)
    return scores

In [ ]:
# This function plots the Confusion matrix with/without normalization + ROC Curve + Precision-Recall Curve
def model_plots(model, X, y, title):
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle(title, fontsize=18)
    plt.subplots_adjust(hspace=0.3)
    axes[0,0].set_title('Confusion Matrix')
    plot_confusion_matrix(model, X, y, display_labels=['No Churn','Churn'], ax=axes[0,0])
    axes[0,1].set_title('Normalized Confusion Matrix')
    plot_confusion_matrix(model, X, y, normalize='true', display_labels=['No Churn','Churn'], ax=axes[0,1])
    axes[1,0].set_title('ROC Curve')
    plot_roc_curve(model, X, y, name='Log.Reg.', ax=axes[1,0])
    axes[1,1].set_title('Precision-Recall Curve')
    plot_precision_recall_curve(model, X, y, name='Log.Reg.', ax=axes[1,1])
    plt.show()

In [ ]:
# This function takes the model and it's related parameters as input and returns the best parameters 
# from given set using Grid Search Algorithm (hyperparameter tuning) 
def hyperparam_gridcv(name, model, params):
    gsearch = GridSearchCV(estimator=model, 
                           param_grid=params, 
                           scoring='f1',
                           cv=5,
                           n_jobs = -1)
    gsearch.fit(X_train, y_train)
    print(name, " - Best Parameters: ", gsearch.best_params_)
    #return gsearch.best_params_

In [ ]:
# This function takes the model and it's related parameters as input and returns the best parameters 
# from given set using Randomized Search Algorithm
def hyperparam_randcv(name, model, params):
    gsearch = RandomizedSearchCV(estimator=model, 
                                 param_distributions=params, 
                                 n_iter = 100,
                                 scoring='f1',
                                 cv=5,
                                 n_jobs = -1)
    gsearch.fit(X_train, y_train)
    print(name, " - Best Parameters: ", gsearch.best_params_)
    #return gsearch.best_params_

### LOGISTIC REGRESSION

In [ ]:
# Fit the base model
logreg = LogisticRegression(max_iter=1000, random_state=10)
logreg.fit(X_train, y_train)

In [ ]:
# Get the training and test set's scores
model_predictions('LOGREG', logreg, X_train, y_train, 'Logistic Reg. Training Set Scores')
model_predictions('LOGREG', logreg, X_test, y_test, 'Logistic Reg. Test Set Scores')
# Plot the confusion matrix, ROC curve and PRC curve
model_plots(logreg, X_test, y_test, "Logistic Regression Results (Base Model)")

* As our data is imbalanced, we will focus on the metrics other than the accuracy. 
* Looking at the results, base model is very poor at predicting Churning Customers as it has a very low precision, recall and f1 score. Looking at the confusion matrix, the model predicts almost half of the Churning customers as wrong and AUC of the Precision-Recall curve is very low.

In [ ]:
# Hyperparameter tuning (it is commented as it takes time)
params_logreg = {'penalty':['l1','l2'], 'C':np.arange(0.1, 3, 0.1)}
#hyperparam_gridcv('Logistic Regression', logreg, params_logreg)

In [ ]:
logreg = LogisticRegression(max_iter=1000, C=2.7, penalty='l2', random_state=10)
logreg.fit(X_train, y_train)

logreg_results = model_predictions('LOGREG', logreg, X_test, y_test, 'Logistic Reg. Test Set Scores')
all_scores = all_scores.append(logreg_results)

model_plots(logreg, X_test, y_test, "Logistic Regression Results (Tuned Model)")

* There is a slight improvement on the predictions but not that much. 

### KNN

In [ ]:
# Fit the base model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
# Get the training and test set's scores
model_predictions('KNN', knn, X_train, y_train, 'KNN Training Set Scores')
model_predictions('KNN', knn, X_test, y_test, 'KNN Test Set Scores')
# Plot the confusion matrix, ROC curve and PRC curve
model_plots(knn, X_test, y_test, "KNN Results (Base Model)")

In [ ]:
# Hyperparameter tuning
params_knn = {'n_neighbors':np.arange(3, 25, 1), 
              'metric':['euclidean','manhattan','minkowski']}
#hyperparam_gridcv('K-Nearest Neighbors', knn, params_knn)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=21, metric='manhattan')
knn.fit(X_train, y_train)

knn_results = model_predictions('KNN', knn, X_test, y_test, 'KNN Test Set Scores')
all_scores = all_scores.append(knn_results)

model_plots(knn, X_test, y_test, "KNN Results (Tuned Model)")

* Hyperparameter tuning improved results of the base model but it is not satisfactory and below the results of Logistic regression. Our model is good at predicting non-churning customers but still poor at churning customers.

### RANDOM FOREST

In [ ]:
# Fit the base model
rf = RandomForestClassifier(random_state=10)
rf.fit(X_train, y_train)

In [ ]:
# Get the training and test set's scores
model_predictions('RND.FOR.', rf, X_train, y_train, 'Random Forest Training Set Scores')
model_predictions('RND.FOR.', rf, X_test, y_test, 'Random Forest Test Set Scores')
# Plot the confusion matrix, ROC curve and PRC curve
model_plots(rf, X_test, y_test, "Random Forest Results (Base Model)")

In [ ]:
# Hyperparameter tuning
params_rf = {'n_estimators':[100, 500, 1000, 1500],
             'max_depth':np.append(np.arange(10, 60, 10), None),
             'class_weight':[None, 'balanced', 'balanced_subsample'],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 5]
            }

# hyperparam_randcv('Random Forest', rf, params_rf)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, 
                            max_depth=10, 
                            max_features='auto', 
                            min_samples_split=5,
                            min_samples_leaf=2, 
                            class_weight='balanced_subsample',
                            random_state=10)
rf.fit(X_train, y_train)

rf_results = model_predictions('RND.FOR.', rf, X_test, y_test, 'Random Forest Test Set Scores')
all_scores = all_scores.append(rf_results)

model_plots(rf, X_test, y_test, "Random Forest Results (Tuned Model)")

In [ ]:
imp = pd.Series(rf.feature_importances_*100, index=X.columns)
imp.sort_values()[-10:].plot(kind='barh', figsize=(10,6))
plt.title('Top 10 Feature Importances of Base Random Forest Model')
plt.xlabel('Percentage Contribution')
plt.show()

##### OBSERVATIONS
* Adjusted model performed slightly better than the base and previous two models in terms of general scores. 


* The most significant improvement is in predicting churning customers. Previous model can predict only half of the churning customers but this model predicts 72% of them correctly.


* From the plot, tenure,total charges, monthly contracts and monthly charges are the most important predictor variables to predict churn which is similar to our observations made before.

### SUPPORT VECTOR MACHINES

In [ ]:
# Fit the base model
svm = SVC(random_state=10)
svm.fit(X_train, y_train)

In [ ]:
# Get the training and test set's scores
model_predictions('SVM', svm, X_train, y_train, 'SVM Training Set Scores')
model_predictions('SVM', svm, X_test, y_test, 'SVM Test Set Scores')
# Plot the confusion matrix, ROC curve and PRC curve
model_plots(svm, X_test, y_test, "SVM Results (Base Model)")

In [ ]:
# Hyperparameter tuning
params_svm = {'C':np.arange(0.1, 3, 0.1), 'kernel':['linear', 'poly', 'rbf']}
#hyperparam_gridcv('SVM', svm, params_svm)

In [ ]:
svm = SVC(C=0.3,
          kernel='linear',
          random_state=10)
svm.fit(X_train, y_train)

svm_results = model_predictions('SVM', svm, X_test, y_test, 'SVM Test Set Scores')
all_scores = all_scores.append(svm_results)

model_plots(svm, X_test, y_test, "SVM Results (Tuned Model)")

* The tuned model has slightly better results than base model but overall it is poor at predicting churning customers like the logistic regression and knn model.

### XGBOOST

In [ ]:
# Fit the base model
xgb = XGBClassifier(random_state=10)
xgb.fit(X_train, y_train)

In [ ]:
# Get the training and test set's scores
model_predictions('XGB', xgb, X_train, y_train, 'XGB Training Set Scores')
model_predictions('XGB', xgb, X_test, y_test, 'XGB Test Set Scores')
# Plot the confusion matrix, ROC curve and PRC curve
model_plots(xgb, X_test, y_test, "XGB Results (Base Model)")

In [ ]:
# Hyperparameter tuning
params_xgb = {'learning_rate': [0.01, 0.05, 0.1],
              'max_depth': [3, 5, 7, 10], 
              'min_child_weight': [1, 5, 10],
              'subsample': [0.5, 0.7, 0.9],
              'colsample_bytree': [0.6, 0.8, 1.0],
              'gamma': [0, 0.05, 0.1],
              'scale_pos_weight':[1,2,3]}
#hyperparam_randcv('XGB', xgb, params_xgb)

In [ ]:
xgb = XGBClassifier(n_estimators=100,
                    learning_rate=0.05,
                    max_depth=5,
                    min_child_weight=10,
                    subsample=0.7,
                    colsample_bytree=0.8,
                    gamma=0.05,
                    scale_pos_weight=2,
                    random_state=10)
xgb.fit(X_train, y_train)

xgb_results = model_predictions('XGB', xgb, X_test, y_test, 'XGB Test Set Scores')
all_scores = all_scores.append(xgb_results)

model_plots(xgb, X_test, y_test, "XGB Results (Tuned Model)")

* This model both has a better accuracy and also predicting the churning customers better than previous models.

### LIGHT GBM

In [ ]:
# Fit the base model
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

In [ ]:
# Get the training and test set's scores
model_predictions('LGBM', lgbm, X_train, y_train, 'LGBM Training Set Scores')
model_predictions('LGBM', lgbm, X_test, y_test, 'LGBM Test Set Scores')
# Plot the confusion matrix, ROC curve and PRC curve
model_plots(lgbm, X_test, y_test, "LGBM Results (Base Model)")

In [ ]:
# Hyperparameter tuning
params_lgbm = {'learning_rate': [0.01, 0.05, 0.1],
               'n_estimators' : [100, 1000, 2000],
               'max_depth': [-1, 3, 5, 7],
               'min_child_weight': [1, 10, 100],
               'subsample': [0.5, 0.7, 0.9],
               'colsample_bytree': [0.6, 0.8, 1.0],
               'reg_alpha': [0, 1, 5, 10],
               'scale_pos_weight':[1, 1.5, 2]}
#hyperparam_randcv('LGBM', lgbm, params_lgbm)

In [ ]:
lgbm = LGBMClassifier(n_estimators=2000,
                    learning_rate=0.05,
                    min_child_weight=100,
                    max_depth=-1,
                    subsample=0.9,
                    colsample_bytree=1,
                    scale_pos_weight=2,
                    reg_alpha=5,
                    random_state=10)
lgbm.fit(X_train, y_train)

lgbm_results = model_predictions('LGBM', lgbm, X_test, y_test, 'LGBM Test Set Scores')
all_scores = all_scores.append(lgbm_results)

model_plots(lgbm, X_test, y_test, "LGBM Results (Tuned Model)")

* The results of LGBM is similar to XGBoost.

# RESULTS

In [ ]:
all_scores.sort_values(by='F1', ascending=False)

* 6 Classification Models are selected, base model is built for each and then these models are improved by hyperparameter tuning.


* Since data is mildly imbalanced (has a churning rate of 26%), most of the models predicted non-churning customers well but couldn't perform same at predicting churning customers. 


* To avoid imbalance problem,
    * Stratified Sampling is applied while splitting
    * Relevent parameters (giving weight to classes) are used while hyperparameter tuning
    
    
* While taking Accuracy score into consideration, F1 score is chosen as the first performance evaluation metric since our data is imbalanced. There was a trade-off between the two, if i tried to increase accuracy of the model, it gave unacceptible low prediction of churning customers, on the other hand if i give weight to churning customer prediction, accuracy score and non-churning customer prediction decreased much.


* In conclusion, the winner of this classification problem is **Light GBM Model** as it has the best F1 score and a good accuracy.